# Import Modules

In [1]:
import os, sys
from bs4 import BeautifulSoup
from itertools import chain
from pytictoc import TicToc
import argparse
import shutil
import re
from pprint import pprint
import pandas as pd
import numpy as np
import datetime as dt
import time
import csv
import json
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import makedir

database = "/home/wmnlab/D/database/"
# database = "/Users/jackbedford/Desktop/MOXA/Code/data/"
dates = [
    "2023-04-01",
]
json_files = [
    "time_sync_lpt3.json",
]
json_files = [os.path.join(database, date, json_file) for date, json_file in zip(dates, json_files)]

exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "_Bandlock_Udp_B3_B7_B8_RM500Q": (6, ["#{:02d}".format(i+1) for i in range(6)]),
    # "_Bandlock_Udp_All_RM500Q": (4, ["#{:02d}".format(i+1) for i in range(4)]),
    # "_Bandlock_Udp_B3_B7_B8_RM500Q": (1, ["#{:02d}".format(i+1) for i in range(1, 2)]),
    # "_Bandlock_Udp_All_RM500Q": (1, ["#{:02d}".format(i+1) for i in range(1, 2)]),
    "_Bandlock_Udp_B1_B3_B7_B8_RM500Q": (6, ["#{:02d}".format(i+1) for i in range(6)]),
    "_Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q": (4, ["#{:02d}".format(i+1) for i in range(4)]),
}
_devices = [
    [
        # "sm00",
        # "sm01",
        # "sm02",
        # "sm03",
        # "sm04",
        # "sm05",
        # "sm06",
        # "sm07",
        # "sm08",
        "qc00",
        "qc01",
        "qc02",
        "qc03",
    ],
    [
        # "sm00",
        # "sm01",
        # "sm02",
        # "sm03",
        # "sm04",
        # "sm05",
        # "sm06",
        # "sm07",
        # "sm08",
        "qc00",
        "qc01",
        "qc02",
        "qc03",
    ],
]
_schemes = [
    ["B1", "B3", "B7", "B8"],
    ["B1B3", "B1B8", "LTE", "All"],
]

class Payload:
    LENGTH = 250              # (Bytes)
    TAG = "000425d401df5e76"  # 2 71828 3 1415926 (hex)            : 8-bytes
    OFS_TIME = (16, 24)       # epoch time of 'yyyy/mm/dd hh:mm:ss': 4-bytes
    OFS_USEC = (24, 32)       # microsecond (usec)                 : 4-bytes
    OFS_SEQN = (32, 40)       # sequence number (start from 1)     : 4-bytes

DATA_RATE = 1000e3  # bits-per-second
PKT_RATE = DATA_RATE / Payload.LENGTH / 8  # packets-per-second
print("packet_rate (pps):", PKT_RATE, "\n")

packet_rate (pps): 500.0 



# mi2log

## Offline Analysis

In [2]:
from mi2log.mi_offline_analysis import mi_decode, error_handling

def fgetter():
    files_collection = []
    tags = "diag_log"
    for filename in filenames:
        if filename.startswith(tags) and filename.endswith(".mi2log"):
            files_collection.append(filename)
    return files_collection

def main():
    files_collection = fgetter()
    if len(files_collection) == 0:
        print("No candidate file.")
    for filename in files_collection:
        fin = os.path.join(source_dir, filename)
        fout = os.path.join(target_dir, "{}.txt".format(filename[:-7]))
        print(">>>>> decode from '{}' into '{}'...".format(fin, fout))
        err_handle = mi_decode(fin, fout)
        err_handles.append(err_handle)
    print()

# ******************************* Check Files *********************************
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        print(os.path.join(database, date, expr))
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
                continue
            
            print("|___", os.path.join(database, date, expr, dev))
            if traces == None:
                # print(os.path.join(database, date, expr, dev))
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            print("|    ", times)
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            if len(traces) != times:
                print("***************************************************************************************")
                print("Warning: the number of traces does not match the specified number of experiment times.")
                print("***************************************************************************************")
            for trace in traces:
                print("|    |___", os.path.join(database, date, expr, dev, trace))
        print()
# *****************************************************************************

# ******************************** Processing *********************************
t = TicToc()  # create instance of class
t.tic()       # Start timer
err_handles = []
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
                continue

            if traces == None:
                print("------------------------------------------")
                print(date, expr, dev)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev)
                target_dir = os.path.join(database, date, expr, dev)
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            for trace in traces:
                print("------------------------------------------")
                print(date, expr, dev, trace)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev, trace, "raw")
                target_dir = os.path.join(database, date, expr, dev, trace, "middle")
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
### Check errors
flag = False
for err_handle in err_handles:
    flag = error_handling(err_handle)
if not flag and err_handles:
    print("**************************************************")
    print("No error occurs!!")
    print("**************************************************")
t.toc()  # Time elapsed since t.tic()
# *****************************************************************************

[INFO] [OfflineReplayer]: Enable LTE_PDCP_UL_Cipher_Data_PDU
[INFO] [OfflineReplayer]: Enable 1xEV_Signaling_Control_Channel_Broadcast
[INFO] [OfflineReplayer]: Enable LTE_PHY_Serv_Cell_Measurement
[INFO] [OfflineReplayer]: Enable LTE_NB1_ML1_Cell_Resel
[INFO] [OfflineReplayer]: Enable LTE_PHY_PDCCH_Decoding_Result
[INFO] [OfflineReplayer]: Enable LTE_PDCP_UL_Config
[INFO] [OfflineReplayer]: Enable LTE_NAS_EMM_OTA_Outgoing_Packet
[INFO] [OfflineReplayer]: Enable GNSS_Glonass_Measurement_Report
[INFO] [OfflineReplayer]: Enable LTE_MAC_Rach_Trigger
[INFO] [OfflineReplayer]: Enable UMTS_NAS_MM_REG_State
[INFO] [OfflineReplayer]: Enable LTE_NB1_ML1_GM_PDSCH_STAT_Ind
[INFO] [OfflineReplayer]: Enable LTE_PDCP_DL_SRB_Integrity_Data_PDU
[INFO] [OfflineReplayer]: Enable LTE_RRC_CDRX_Events_Info
[INFO] [OfflineReplayer]: Enable 5G_NR_LL1_FW_Serving_FTL
[INFO] [OfflineReplayer]: Enable LTE_PHY_PUSCH_Tx_Report
[INFO] [OfflineReplayer]: Enable 5G_NR_RLC_DL_Stats
[INFO] [OfflineReplayer]: Enable 5G_

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03 does not exist.

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.

/home/

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:52.147093:LteMeasurementAnalyzer RSRP=-97.1875dBm RSRQ=-11.6875dB # of neighbors=2
    Cell_ID=362 RSRP=-97.75dBm RSRQ=-12.625dB
    Cell_ID=370 RSRP=-104.8125dBm RSRQ=-21.4375dB

[INFO] [LteRrcAnalyzer]: MEAS_PCELL: {'timestamp': '2023-03-26 08:46:52.147590', 'rsrp': 43, 'rssi': -98, 'rsrq': 16}
[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:52.208140:LteMeasurementAnalyzer RSRP=-97.125dBm RSRQ=-11.6875dB # of neighbors=2
    Cell_ID=362 RSRP=-97.5625dBm RSRQ=-12.4375dB
    Cell_ID=370 RSRP=-105.0dBm RSRQ=-21.5625dB

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:52.282086:LteMeasurementAnalyzer RSRP=-97.0625dBm RSRQ=-11.6875dB # of neighbors=2
    Cell_ID=362 RSRP=-97.5625dBm RSRQ=-12.5625dB
    Cell_ID=370 RSRP=-104.625dBm RSRQ=-21.6875dB

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:52.984203:LteMeasurementAnalyzer RSRP=-97.0625dBm RSRQ=-11.3125dB # of neighbors=2
    Cell_ID=362 RSRP=-97.0dBm RSRQ=-12.875dB
    Cell_ID=3

(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor C

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:56.824192:LteMeasurementAnalyzer RSRP=-97.6875dBm RSRQ=-14.125dB # of neighbors=1
    Cell_ID=362 RSRP=-96.4375dBm RSRQ=-14.75dB

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:56.827068:LteMeasurementAnalyzer RSRP=-93.375dBm RSRQ=-22.5625dB # of neighbors=3
    Cell_ID=362 RSRP=-84.9375dBm RSRQ=-8.4375dB
    Cell_ID=370 RSRP=-91.4375dBm RSRQ=-19.625dB
    Cell_ID=286 RSRP=-93.125dBm RSRQ=-22.125dB

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:57.787074:LteMeasurementAnalyzer RSRP=-98.3125dBm RSRQ=-20.9375dB # of neighbors=3
    Cell_ID=362 RSRP=-81.3125dBm RSRQ=-7.25dB
    Cell_ID=370 RSRP=-96.6875dBm RSRQ=-18.9375dB
    Cell_ID=286 RSRP=-98.3125dBm RSRQ=-22.5dB

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:58.104189:LteMeasurementAnalyzer RSRP=-96.0625dBm RSRQ=-13.375dB # of neighbors=1
    Cell_ID=362 RSRP=-96.3125dBm RSRQ=-14.1875dB

[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:46:58.107067:LteMeasurementAnalyzer RSRP

...

(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor C

[INFO] [LteRrcAnalyzer]: HANDOVER 2023-03-26 08:51:28.474297 from (3650, 178) to (3650, 394)
[INFO] [LteRrcAnalyzer]: LteRrcStatus cellID=394 frequency=3650 TAC=None connected=False
[INFO] [LteRrcAnalyzer]: UPDATA_NR_CELL 2023-03-26 08:51:28.474297 (631000, 394)
[INFO] [LteMeasurementAnalyzer]: 2023-03-26 08:51:28.515123:LteMeasurementAnalyzer RSRP=-105.25dBm RSRQ=-21.1875dB # of neighbors=4
    Cell_ID=386 RSRP=-101.9375dBm RSRQ=-17.4375dB
    Cell_ID=178 RSRP=-101.9375dBm RSRQ=-16.875dB
    Cell_ID=122 RSRP=-102.5dBm RSRQ=-17.5dB
    Cell_ID=343 RSRP=-107.1875dBm RSRQ=-21.0dB

[INFO] [LteRrcAnalyzer]: RRC_RECONFIG: LteRrcConfig
LteRrcStatus cellID=394 frequency=3650 TAC=None connected=FalseLteRrcSibServ 7 0 inf 0
LteRrcSibIntraFreqConfig None None None None
LteMeasObjectEutra 1 3650 0 
[INFO] [LteRrcAnalyzer]: RRC_RECONFIG: LteRrcConfig
LteRrcStatus cellID=394 frequency=3650 TAC=None connected=FalseLteRrcSibServ 7 0 inf 0
LteRrcSibIntraFreqConfig None None None None
LteMeasObjectEutr

(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor C

[INFO] [LteRrcAnalyzer]: HANDOVER 2023-03-26 08:51:33.679229 from (3650, 394) to (3650, 122)
[INFO] [LteRrcAnalyzer]: LteRrcStatus cellID=122 frequency=3650 TAC=None connected=False
[INFO] [LteRrcAnalyzer]: UPDATA_NR_CELL 2023-03-26 08:51:33.679229 (631000, 394)
[INFO] [LteRrcAnalyzer]: SIB_CONFIG: LteRrcConfig
LteRrcStatus cellID=122 frequency=3650 TAC=None connected=FalseLteRrcSibServ 5 6 6.0 4
LteRrcSibIntraFreqConfig 2 0 23 62.0
LteRrcSibInterFreqConfig LTE 1400 2 -124 23 5 8 14
LteMeasObjectEutra 1 3650 0 LteMeasObjectEutra 2 1750 0 LteReportConfig 1 1.0 a3 2.0 NoneLteReportConfig 2 2.0 a2 -110 NoneLteReportConfig 3 1.0 a2 -140 NoneLteReportConfig 4 0.0MeasObj 1 (1, 1)
MeasObj 2 (1, 2)
MeasObj 3 (2, 3)
MeasObj 4 (1, 4)

[INFO] [LteRrcAnalyzer]: SIB_CONFIG: LteRrcConfig
LteRrcStatus cellID=122 frequency=3650 TAC=None connected=FalseLteRrcSibServ 5 6 6.0 4
LteRrcSibIntraFreqConfig 2 0 23 62.0
LteRrcSibInterFreqConfig LTE 1400 2 -124 23 5 8 14
LteMeasObjectEutra 1 3650 0 LteMeasObjec

(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40
(MI)Unknown LTE PHY Connected Neighbor Cell Meas subpkt id and version: 31 - 40

**************************************************
No error occurs!!
**************************************************
Elapsed time is 28.463221 seconds.


## xml to csv

In [3]:
from mi2log.xml_mi import xml_to_csv_rrc, xml_to_csv_ml1, xml_to_csv_nr_ml1

def fgetter():
    files_collection = []
    tags = "diag_log"
    for filename in filenames:
        if filename.startswith(tags) and filename.endswith(".txt"):
            files_collection.append(filename)
    return files_collection

def main():
    files_collection = fgetter()
    if len(files_collection) == 0:
        print("No candidate file.")
    for filename in files_collection:
        fin = os.path.join(source_dir, filename)
        fout1 = os.path.join(target_dir, "{}_rrc.csv".format(filename[:-4]))
        fout2 = os.path.join(target_dir, "{}_ml1.csv".format(filename[:-4]))
        fout3 = os.path.join(target_dir, "{}_nr_ml1.csv".format(filename[:-4]))
        print(">>>>> convert from '{}' into '{}'...".format(fin, fout1))
        xml_to_csv_rrc(fin, fout1)
        # savemove(os.path.join(source_dir, "{}_rrc.csv".format(filename[:-4])), target_dir, "{}_rrc.csv".format(filename[:-4]))
        print(">>>>> convert from '{}' into '{}'...".format(fin, fout2))
        xml_to_csv_ml1(fin, fout2)
        # savemove(os.path.join(source_dir, "{}_ml1.csv".format(filename[:-4])), target_dir, "{}_ml1.csv".format(filename[:-4]))
        print(">>>>> convert from '{}' into '{}'...".format(fin, fout3))
        xml_to_csv_nr_ml1(fin, fout3)
        # savemove(os.path.join(source_dir, "{}_nr_ml1.csv".format(filename[:-4])), target_dir, "{}_nr_ml1.csv".format(filename[:-4]))
    print()

# ******************************* Check Files *********************************
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        print(os.path.join(database, date, expr))
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
                continue
            
            print("|___", os.path.join(database, date, expr, dev))
            if traces == None:
                # print(os.path.join(database, date, expr, dev))
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            print("|    ", times)
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            if len(traces) != times:
                print("***************************************************************************************")
                print("Warning: the number of traces does not match the specified number of experiment times.")
                print("***************************************************************************************")
            for trace in traces:
                print("|    |___", os.path.join(database, date, expr, dev, trace))
        print()
# *****************************************************************************

# ******************************** Processing *********************************
t = TicToc()  # create instance of class
t.tic()       # Start timer
err_handles = []
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
                continue

            if traces == None:
                print("------------------------------------------")
                print(date, expr, dev)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev)
                target_dir = os.path.join(database, date, expr, dev)
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            for trace in traces:
                print("------------------------------------------")
                print(date, expr, dev, trace)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev, trace, "middle")
                target_dir = os.path.join(database, date, expr, dev, trace, "data")
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
t.toc()  # Time elapsed since t.tic()
# *****************************************************************************

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03 does not exist.

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.

/home/

# Pcap

## pcap to csv

In [4]:
from udp.pcap_to_csv import pcap_to_csv

def fgetter():
    files_collection = []
    tags = ("server_pcap", "client_pcap")
    for filename in filenames:
        if filename.startswith(tags) and filename.endswith(".pcap"):
            files_collection.append(filename)
    return files_collection

def main():
    files_collection = fgetter()
    if len(files_collection) == 0:
        print("No candidate file.")
    for filename in files_collection:
        fin = os.path.join(source_dir, filename)
        fout = os.path.join(target_dir, "{}.csv".format(filename[:-5]))
        print(">>>>> convert from '{}' into '{}'...".format(fin, fout))
        err_handle = pcap_to_csv(fin, fout)
        err_handles.append(err_handle)
    print()

# ******************************* Check Files *********************************
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        print(os.path.join(database, date, expr))
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
                continue
            
            print("|___", os.path.join(database, date, expr, dev))
            if traces == None:
                # print(os.path.join(database, date, expr, dev))
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            print("|    ", times)
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            if len(traces) != times:
                print("***************************************************************************************")
                print("Warning: the number of traces does not match the specified number of experiment times.")
                print("***************************************************************************************")
            for trace in traces:
                print("|    |___", os.path.join(database, date, expr, dev, trace))
        print()
# *****************************************************************************

# ******************************** Processing *********************************
t = TicToc()  # create instance of class
t.tic()       # Start timer
err_handles = []
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
                continue

            if traces == None:
                print("------------------------------------------")
                print(date, expr, dev)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev)
                target_dir = os.path.join(database, date, expr, dev)
                makedir(target_dir)
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
                # filenames = os.listdir(source_dir)
                # main()
                # continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            for trace in traces:
                print("------------------------------------------")
                print(date, expr, dev, trace)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev, trace, "raw")
                target_dir = os.path.join(database, date, expr, dev, trace, "middle")
                if expr == "tsync":
                    source_dir = os.path.join(database, date, expr, dev, trace)
                    target_dir = os.path.join(database, date, expr, dev, trace)
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
### Check errors
flag = False
for err_handle in err_handles:
    flag = error_handling(err_handle)
if not flag and err_handles:
    print("**************************************************")
    print("No error occurs!!")
    print("**************************************************")
t.toc()  # Time elapsed since t.tic()
# *****************************************************************************

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03 does not exist.

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.

/home/

## parse packet info

In [5]:
from udp.parse_info import filter, parse_packet_info, parse_brief_info

def fgetter(direction, terminal):
    if direction == "uplink" and terminal == "client":
        tags = ("client_pcap_BL", "client_pcap_UL")
    elif direction == "downlink" and terminal == "client":
        tags = ("client_pcap_BL", "client_pcap_DL")
    elif direction == "uplink" and terminal == "server":
        tags = ("server_pcap_BL", "server_pcap_UL")
    elif direction == "downlink" and terminal == "server":
        tags = ("server_pcap_BL", "server_pcap_DL")
    for filename in filenames:
        if filename.startswith(tags) and filename.endswith(".csv"):
            print(">>>>>", os.path.join(source_dir, filename))
            return os.path.join(source_dir, filename)
    print("No candidate file.")
    return None

def main():
    ### detailed information for each udp packet's frame
    # udp_uplk_client_pkt_info   # udp ultx detailed info
    # udp_uplk_server_pkt_info   # udp ulrx detailed info
    # udp_dnlk_server_pkt_info   # udp dltx detailed info
    # udp_dnlk_client_pkt_info   # udp dlrx detailed info

    ### brief information for each udp packet
    # udp_uplk_client_pkt_brief  # udp ultx brief info
    # udp_uplk_server_pkt_brief  # udp ulrx brief info
    # udp_dnlk_server_pkt_brief  # udp dltx brief info
    # udp_dnlk_client_pkt_brief  # udp dlrx brief info

    ### dirpath
    # source_dir
    # target_dir

    ### parse detailed information & brief information for each packet
    
    ## client_pcap_BL or client_pcap_UL
    filepath = fgetter("uplink", "client")
    if filepath:
        t1 = TicToc()  # create instance of class
        t1.tic()       # Start timer
        df = pd.read_csv(filepath, sep='@')
        df_ultx = filter(df.copy(), "uplink", "client", "udp")
        df_ultx = parse_packet_info(df_ultx, os.path.join(target_dir, "udp_uplk_client_pkt_info.csv"))
        df_ultx = parse_brief_info(df_ultx, os.path.join(target_dir, "udp_uplk_client_pkt_brief.csv"))
        os.system(f'rm {os.path.join(target_dir, "udp_uplk_client_pkt_info.csv")}') # Remove
        t1.toc()  # Time elapsed since t1.tic()
    
    ## client_pcap_BL or client_pcap_DL
    filepath = fgetter("downlink", "client")
    if filepath:
        t1 = TicToc()  # create instance of class
        t1.tic()       # Start timer
        df = pd.read_csv(filepath, sep='@')
        df_dlrx = filter(df.copy(), "downlink", "client", "udp")
        df_dlrx = parse_packet_info(df_dlrx, os.path.join(target_dir, "udp_dnlk_client_pkt_info.csv"))
        df_dlrx = parse_brief_info(df_dlrx, os.path.join(target_dir, "udp_dnlk_client_pkt_brief.csv"))
        os.system(f'rm {os.path.join(target_dir, "udp_dnlk_client_pkt_info.csv")}') # Remove
        t1.toc()  # Time elapsed since t1.tic()

    ## server_pcap_BL or server_pcap_UL
    filepath = fgetter("uplink", "server")
    if filepath:
        t1 = TicToc()  # create instance of class
        t1.tic()       # Start timer
        df = pd.read_csv(filepath, sep='@')
        df_ulrx = filter(df.copy(), "uplink", "server", "udp")
        df_ulrx = parse_packet_info(df_ulrx, os.path.join(target_dir, "udp_uplk_server_pkt_info.csv"))
        df_ulrx = parse_brief_info(df_ulrx, os.path.join(target_dir, "udp_uplk_server_pkt_brief.csv"))
        os.system(f'rm {os.path.join(target_dir, "udp_uplk_server_pkt_info.csv")}') # Remove
        t1.toc()  # Time elapsed since t1.tic()
    
    ## server_pcap_BL or server_pcap_DL
    filepath = fgetter("downlink", "server")
    if filepath:
        t1 = TicToc()  # create instance of class
        t1.tic()       # Start timer
        df = pd.read_csv(filepath, sep='@')
        df_dltx = filter(df.copy(), "downlink", "server", "udp")
        df_dltx = parse_packet_info(df_dltx, os.path.join(target_dir, "udp_dnlk_server_pkt_info.csv"))
        df_dltx = parse_brief_info(df_dltx, os.path.join(target_dir, "udp_dnlk_server_pkt_brief.csv"))
        os.system(f'rm {os.path.join(target_dir, "udp_dnlk_server_pkt_info.csv")}') # Remove
        t1.toc()  # Time elapsed since t1.tic()
    print()
    return

# ******************************* Check Files *********************************
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        print(os.path.join(database, date, expr))
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
                continue
            
            print("|___", os.path.join(database, date, expr, dev))
            if traces == None:
                # print(os.path.join(database, date, expr, dev))
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            print("|    ", times)
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            if len(traces) != times:
                print("***************************************************************************************")
                print("Warning: the number of traces does not match the specified number of experiment times.")
                print("***************************************************************************************")
            for trace in traces:
                print("|    |___", os.path.join(database, date, expr, dev, trace))
        print()
# *****************************************************************************

# ******************************** Processing *********************************
t = TicToc()  # create instance of class
t.tic()       # Start timer
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
                continue

            if traces == None:
                print("------------------------------------------")
                print(date, expr, dev)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev)
                target_dir = os.path.join(database, date, expr, dev)
                makedir(target_dir)
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
                # filenames = os.listdir(source_dir)
                # main()
                # continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            for trace in traces:
                print("------------------------------------------")
                print(date, expr, dev, trace)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev, trace, "middle")
                target_dir = os.path.join(database, date, expr, dev, trace, "middle")
                if expr == "tsync":
                    source_dir = os.path.join(database, date, expr, dev, trace)
                    target_dir = os.path.join(database, date, expr, dev, trace)
                makedir(target_dir)
                filenames = os.listdir(source_dir)
                main()
t.toc()  # Time elapsed since t.tic()
# *****************************************************************************


/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03 does not exist.

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.

/home/

100%|██████████| 143981/143981 [00:15<00:00, 9298.36it/s] 


output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/middle/udp_uplk_client_pkt_info.csv


100%|██████████| 143981/143981 [00:05<00:00, 24875.47it/s]


output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/middle/udp_uplk_client_pkt_brief.csv
Elapsed time is 40.655969 seconds.
>>>>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/middle/client_pcap_BL_qc00_3270_3271_2023-03-26_16-46-44_sock.csv


...

output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/middle/udp_uplk_server_pkt_brief.csv
Elapsed time is 41.298013 seconds.
>>>>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/middle/server_pcap_BL_qc03_3276_3277_2023-03-26_16-46-47_sock.csv


100%|██████████| 146457/146457 [00:15<00:00, 9502.80it/s]


output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/middle/udp_dnlk_server_pkt_info.csv


100%|██████████| 146457/146457 [00:06<00:00, 24119.18it/s]


output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/middle/udp_dnlk_server_pkt_brief.csv
Elapsed time is 40.773854 seconds.

Elapsed time is 487.988406 seconds.


## parse loss latency

In [6]:
from udp.parse_loss_latency import get_loss, consolidate, compensate, get_latency, get_statistics

t = TicToc()  # create instance of class
t.tic()  # Start timer
# --------------------- (3) decode a batch of files (User Settings) ---------------------
# err_handles = []
for date in dates:
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        print(os.path.join(database, date, expr))
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("|___ {} does not exist.".format(os.path.join(database, date, expr, dev)))
                continue
            
            print("|___", os.path.join(database, date, expr, dev))
            if traces == None:
                # print(os.path.join(database, date, expr, dev))
                continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            print("|    ", times)
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            if len(traces) != times:
                print("***************************************************************************************")
                print("Warning: the number of traces does not match the specified number of experiment times.")
                print("***************************************************************************************")
            for trace in traces:
                print("|    |___", os.path.join(database, date, expr, dev, trace))
        print()
        
# --------------------- Phase 2: Parse packet loss & latency --------------------- 
### Read files
for date, json_file in zip(dates, json_files):
    for (expr, (times, traces)), devices in zip(exps.items(), _devices):
        for dev in devices:
            if not os.path.isdir(os.path.join(database, date, expr, dev)):
                print("{} does not exist.\n".format(os.path.join(database, date, expr, dev)))
                continue

            if traces == None:
                print("------------------------------------------")
                print(date, expr, dev)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev)
                target_dir = os.path.join(database, date, expr, dev)
                makedir(target_dir)
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
                # filenames = os.listdir(source_dir)
                # main()
                # continue
            elif len(traces) == 0:
                traces = sorted(os.listdir(os.path.join(database, date, expr, dev)))
            
            traces = [trace for trace in traces if os.path.isdir(os.path.join(database, date, expr, dev, trace))]
            for trace in traces:
                print("------------------------------------------")
                print(date, expr, dev, trace)
                print("------------------------------------------")
                source_dir = os.path.join(database, date, expr, dev, trace, "middle")
                target_dir1 = os.path.join(database, date, expr, dev, trace, "data")
                target_dir2 = os.path.join(database, date, expr, dev, trace, "statistics")
                if expr == "tsync":
                    source_dir = os.path.join(database, date, expr, dev, trace)
                    target_dir = os.path.join(database, date, expr, dev, trace)
                makedir(target_dir1)
                makedir(target_dir2)
                filenames = os.listdir(source_dir)
                t1 = TicToc()  # create instance of class
                t1.tic()  # Start timer
                
                dl_txdf = pd.read_csv(os.path.join(source_dir, "udp_dnlk_server_pkt_brief.csv"))
                dl_rxdf = pd.read_csv(os.path.join(source_dir, "udp_dnlk_client_pkt_brief.csv"))
                ul_txdf = pd.read_csv(os.path.join(source_dir, "udp_uplk_client_pkt_brief.csv"))
                ul_rxdf = pd.read_csv(os.path.join(source_dir, "udp_uplk_server_pkt_brief.csv"))
                
                dl_txseq = list(dl_txdf["seq"].array)
                dl_rxseq = list(dl_rxdf["seq"].array)
                dlst = max(dl_txseq[0], dl_rxseq[0])
                dlet = min(dl_txseq[-1], dl_rxseq[-1])
                # print(dlst, dlet)

                ul_txseq = list(ul_txdf["seq"].array)
                ul_rxseq = list(ul_rxdf["seq"].array)
                ulst = max(ul_txseq[0], ul_rxseq[0])
                ulet = min(ul_txseq[-1], ul_rxseq[-1])
                # print(ulst, ulet)

                st = max(dlst, ulst)
                et = min(dlet, ulet)
                # print("----------------")
                st += PKT_RATE * 5  # 開頭切5秒
                et -= PKT_RATE * 5  # 結尾切5秒
                # print(st, et)

                dl_txdf = dl_txdf[(dl_txdf["seq"] >= st) & (dl_txdf["seq"] <= et)].copy().reset_index(drop=True)
                dl_rxdf = dl_rxdf[(dl_rxdf["seq"] >= st) & (dl_rxdf["seq"] <= et)].copy().reset_index(drop=True)
                ul_txdf = ul_txdf[(ul_txdf["seq"] >= st) & (ul_txdf["seq"] <= et)].copy().reset_index(drop=True)
                ul_rxdf = ul_rxdf[(ul_rxdf["seq"] >= st) & (ul_rxdf["seq"] <= et)].copy().reset_index(drop=True)
                
                json_object = {}
                if os.path.isfile(json_file):
                    with open(json_file, 'r') as f:
                        json_object = json.load(f)
                else:
                    print('*************', json_file, 'does not exist! *************')
                delta = pd.DataFrame.from_dict(json_object, orient='index', columns=['delta']).reset_index(names='Timestamp')
                delta['Timestamp'] = pd.to_datetime(delta['Timestamp'])
                delta['timedelta'] = pd.to_timedelta(delta['delta'], unit='seconds')

                ### Downlink
                fout1_dl = os.path.join(target_dir1, "udp_dnlk_loss_latency.csv")
                fout2_dl = os.path.join(target_dir2, "udp_dnlk_loss_statistics.csv")
                fout3_dl = os.path.join(target_dir2, "udp_dnlk_excl_statistics.csv")
                
                losdf = get_loss(dl_rxdf.copy(), dl_txdf.copy())
                latdf = consolidate(dl_rxdf.copy(), dl_txdf.copy())
                df = pd.concat([latdf, losdf], axis=0)
                df = df.sort_values(by=["seq"]).reset_index(drop=True)
                df = compensate(df.copy(), "dl", delta.copy())
                df = get_latency(df.copy(), "dl")
                get_statistics(df.copy(), fout1_dl, fout2_dl, fout3_dl)
                
                ### Uplink
                fout1_ul = os.path.join(target_dir1, "udp_uplk_loss_latency.csv")
                fout2_ul = os.path.join(target_dir2, "udp_uplk_loss_statistics.csv")
                fout3_ul = os.path.join(target_dir2, "udp_uplk_excl_statistics.csv")
                
                losdf = get_loss(ul_rxdf.copy(), ul_txdf.copy())
                latdf = consolidate(ul_rxdf.copy(), ul_txdf.copy())
                df = pd.concat([latdf, losdf], axis=0)
                df = df.sort_values(by=["seq"]).reset_index(drop=True)
                df = compensate(df.copy(), "ul", delta.copy())
                df = get_latency(df.copy(), "ul")
                get_statistics(df.copy(), fout1_ul, fout2_ul, fout3_ul)
                
                t1.toc()
# ### Check errors
# flag = False
# for err_handle in err_handles:
#     flag = error_handling(err_handle)
# if not flag:
#     print("**************************************************")
#     print("No error occurs!!")
#     print("**************************************************")
t.toc()  # Time elapsed since t.tic()


packet_rate (pps): 500.0 

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02 does not exist.
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03 does not exist.

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02
|___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03
|     1
|    |___ /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02

/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/q

100%|██████████| 138191/138191 [00:00<00:00, 527145.14it/s]


2.3308606907030147 seconds
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/data/udp_dnlk_loss_latency.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/statistics/udp_dnlk_loss_statistics.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/statistics/udp_dnlk_excl_statistics.csv
------------------------------------------
min latency:          0.0033 seconds
max latency:          0.449985 seconds
mean latency:         0.010501 seconds
stdev latency:        0.018395 seconds
jitter:               0.002698 seconds
negative latency:     0
total_packet_sent:    138981
total_packet_recv:    138191
total_packet_loss:    790
packet_loss_rate(%):  0.568 %
total_excs_latency:   1044
excs_latency_rate(%): 0.755 %
experiment_time(sec): 277.97299 seconds
------------------------------------------



100%|██████████| 138748/138748 [00:00<00:00, 460109.29it/s]


2.3314909545969575 seconds
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/data/udp_uplk_loss_latency.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/statistics/udp_uplk_loss_statistics.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc00/#02/statistics/udp_uplk_excl_statistics.csv
------------------------------------------
min latency:          0.003849 seconds
max latency:          0.583018 seconds
mean latency:         0.011683 seconds
stdev latency:        0.020781 seconds
jitter:               0.001638 seconds
negative latency:     0
total_packet_sent:    138981
total_packet_recv:    138748
total_packet_loss:    233
packet_loss_rate(%):  0.168 %
total_excs_latency:   469
excs_latency_rate(%): 0.338 %
experiment_time(sec): 277.972989 seconds
------------------------------------------

Elapsed time is 7.704582 seconds.
------------------------------------------
2023-03-26_#1 _Band

...

2.3314909545969575 seconds
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02/data/udp_uplk_loss_latency.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02/statistics/udp_uplk_loss_statistics.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc02/#02/statistics/udp_uplk_excl_statistics.csv
------------------------------------------
min latency:          0.003669 seconds
max latency:          0.474197 seconds
mean latency:         0.012281 seconds
stdev latency:        0.019571 seconds
jitter:               0.001618 seconds
negative latency:     0
total_packet_sent:    138981
total_packet_recv:    138798
total_packet_loss:    183
packet_loss_rate(%):  0.132 %
total_excs_latency:   682
excs_latency_rate(%): 0.491 %
experiment_time(sec): 277.972989 seconds
------------------------------------------

Elapsed time is 7.346247 seconds.
------------------------------------------
2023-03-26_#1 _Band

100%|██████████| 136149/136149 [00:00<00:00, 497747.52it/s]


2.3308606907030147 seconds
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/data/udp_dnlk_loss_latency.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/statistics/udp_dnlk_loss_statistics.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/statistics/udp_dnlk_excl_statistics.csv
------------------------------------------
min latency:          0.003509 seconds
max latency:          0.408661 seconds
mean latency:         0.010001 seconds
stdev latency:        0.013466 seconds
jitter:               0.002694 seconds
negative latency:     0
total_packet_sent:    138889
total_packet_recv:    136149
total_packet_loss:    2740
packet_loss_rate(%):  1.973 %
total_excs_latency:   696
excs_latency_rate(%): 0.511 %
experiment_time(sec): 277.788981 seconds
------------------------------------------



100%|██████████| 138727/138727 [00:00<00:00, 467447.12it/s]


2.3314909545969575 seconds
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/data/udp_uplk_loss_latency.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/statistics/udp_uplk_loss_statistics.csv
output >>> /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_All_RM500Q/qc03/#02/statistics/udp_uplk_excl_statistics.csv
------------------------------------------
min latency:          0.003898 seconds
max latency:          1.465742 seconds
mean latency:         0.017836 seconds
stdev latency:        0.075922 seconds
jitter:               0.00166 seconds
negative latency:     0
total_packet_sent:    138889
total_packet_recv:    138727
total_packet_loss:    162
packet_loss_rate(%):  0.117 %
total_excs_latency:   1862
excs_latency_rate(%): 1.342 %
experiment_time(sec): 277.78898 seconds
------------------------------------------

Elapsed time is 7.302325 seconds.
Elapsed time is 22.354672 seconds.


## generate combo

In [7]:
for date in dates:
    for (expr, (times, traces)), schemes in zip(exps.items(), _schemes):
        for trace in traces:
            target_dir = os.path.join(database, date, expr, "combo", trace)
            makedir(target_dir)
            for tag in ["dnlk", "uplk"]:
            # for tag in ["dnlk",]:
            # for tag in ["uplk",]:
                print("------------------------------------------")
                print(date, expr, trace, tag)  
                print("------------------------------------------")
                t = TicToc()
                t.tic()
                dfs = []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    source_dir = os.path.join(database, date, expr, dev, trace, "data")
                    dfs.append(pd.read_csv(os.path.join(source_dir, f"udp_{tag}_loss_latency.csv")))
                ### TODO 1
                st, et = [], []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    st.append(dfs[i]['seq'].array[0])
                    et.append(dfs[i]['seq'].array[-1])
                st, et = max(st), min(et)
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    dfs[i] = dfs[i][(dfs[i]['seq'] >= st) & (dfs[i]['seq'] <= et)]
                    dfs[i].reset_index(drop=True, inplace=True)
                df = dfs[0][['seq', 'Timestamp']]
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    dfs[i] = dfs[i][['xmit_time','arr_time','lost','excl','latency']]
                    dfs[i].rename(
                        columns={
                            'xmit_time': f'xmit_time_{scheme}',
                            'arr_time': f'arr_time_{scheme}',
                            'lost': f'lost_{scheme}',
                            'excl': f'excl_{scheme}',
                            'latency': f'latency_{scheme}',
                        }, inplace=True
                    )
                df = pd.concat([df, *dfs], axis=1)
                ### TODO 2
                xs = list(it.combinations(range(len(schemes)), 2))
                for x in xs:
                    df[f'lost_{schemes[x[0]]}+{schemes[x[1]]}'] = df[f'lost_{schemes[x[0]]}'] & df[f'lost_{schemes[x[1]]}']
                    df[f'excl_{schemes[x[0]]}+{schemes[x[1]]}'] = df[f'excl_{schemes[x[0]]}'] & df[f'excl_{schemes[x[1]]}']
                    df[f'latency_{schemes[x[0]]}+{schemes[x[1]]}'] = df[[f'latency_{schemes[x[0]]}', f'latency_{schemes[x[1]]}']].min(axis=1)
                fout1 = os.path.join(target_dir, f"udp_{tag}_combo_loss_latency.csv")
                print("output >>>", fout1)
                df.to_csv(fout1, index=False)
                ### TODO 3
                colnames = []
                data = []
                print('', 'loss(%)', 'excl(%)', 'latency', 'max_latency', sep='\t')
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    _df = df[df[f'lost_{scheme}'] == False]
                    loss = sum(df[f'lost_{scheme}']) / (len(df)+1e-9) * 100
                    excl = sum(_df[f'excl_{scheme}']) / (len(_df)+1e-9) * 100
                    latency = round(mean(_df[f'latency_{scheme}']), 6)
                    max_lat = round(max(_df[f'latency_{scheme}']), 6)
                    min_lat = round(min(_df[f'latency_{scheme}']), 6)
                    data = [*data, *[loss, excl, latency, max_lat]]
                    colnames = [*colnames, *[f'lost_{scheme}', f'excl_{scheme}', f'latency_{scheme}', f'max_lat_{scheme}', f'min_lat_{scheme}']]
                    # print(scheme, round(loss, 3), round(excl, 3), latency, max_lat, sep='\t')
                    print(scheme, loss, excl, latency, max_lat, min_lat, sep='\t')
                for x in xs:
                    _df = df[df[f'lost_{schemes[x[0]]}+{schemes[x[1]]}'] == False]
                    loss = sum(df[f'lost_{schemes[x[0]]}+{schemes[x[1]]}']) / (len(df)+1e-9) * 100
                    excl = sum(_df[f'excl_{schemes[x[0]]}+{schemes[x[1]]}']) / (len(_df)+1e-9) * 100
                    latency = round(mean(_df[f'latency_{schemes[x[0]]}+{schemes[x[1]]}']), 6)
                    max_lat = round(max(_df[f'latency_{schemes[x[0]]}+{schemes[x[1]]}']), 6)
                    min_lat = round(max(_df[f'latency_{schemes[x[0]]}+{schemes[x[1]]}']), 6)
                    data = [*data, *[loss, excl, latency, max_lat]]
                    colnames = [*colnames, *[f'lost_{schemes[x[0]]}+{schemes[x[1]]}', f'excl_{schemes[x[0]]}+{schemes[x[1]]}', f'latency_{schemes[x[0]]}+{schemes[x[1]]}', f'max_lat_{schemes[x[0]]}+{schemes[x[1]]}', f'min_lat_{schemes[x[0]]}+{schemes[x[1]]}']]
                    # print(f'{schemes[x[0]]}+{schemes[x[1]]}', round(loss, 3), round(excl, 3), latency, max_lat, sep='\t')
                    print(f'{schemes[x[0]]}+{schemes[x[1]]}', loss, excl, latency, max_lat, min_lat, sep='\t')
                fout2 = os.path.join(target_dir, f"udp_{tag}_combo_statistics.csv")
                print("output >>>", fout2)
                with open(fout2, "w", newline='') as fp:
                    writer = csv.writer(fp)
                    writer.writerow(colnames)
                    writer.writerow(data)
                t.toc()
                ### TODO END

mkdir /home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#02
------------------------------------------
2023-03-26_#1 _Bandlock_Udp_B3_B7_B8_RM500Q #02 dnlk
------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: '/home/wmnlab/D/database/2023-03-26_#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc00/#02/data/udp_dnlk_loss_latency.csv'